### Imports

In [3]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Embedding, LSTM
from sklearn.model_selection import train_test_split
import scipy.signal
import numpy as np
import pandas
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from matplotlib.pyplot import figure
import random
from heapq import nlargest

### Data Load

In [4]:
# data = pandas.read_csv('./data/Learning/brake, acc/Train_Set.csv')
# test = pandas.read_csv('./data/Learning/brake, acc/Test_Set.csv')
data = pandas.read_csv('./data/Learning/MyData/version1/Train_Set.csv')
test = pandas.read_csv('./data/Learning/MyData/version1/Test_Set.csv')
# data = pandas.read_csv('./data/Learning/MyData/version2/Train_Set.csv')
# test = pandas.read_csv('./data/Learning/MyData/version2/Test_Set.csv')
# data = pandas.read_csv('./data/Learning/MyData/version3/Train_Set.csv')
# test = pandas.read_csv('./data/Learning/MyData/version3/Test_Set.csv')


# data = data[0:106092]
# data = data[0:105658]
x_train_set = data[['x','y','z']]
# x_train_set = data[['x']]

target = data['AccStatus']

x_test_set = test[['x','y','z']]
# x_test_set = test[['x']]
y_test_set = test['AccStatus']


x_train_set = np.array(x_train_set,dtype=float)          
target = np.array(target,dtype=float)

x_test_set =  np.array(x_test_set,dtype=float)
y_test_set =  np.array(y_test_set,dtype=float)

In [8]:
len(x_test_set)

34427

In [6]:
def SmoothData(data):
    size = data.shape[0] 
    dim = data.shape[1]
    
    smooth_data = np.zeros((size,dim))
    
    smooth_data[0] = data[0]
    smooth_data[1] = data[1]
    smooth_data[size-1] = data[size-1]
    smooth_data[size-2] = data[size-2]
    
    for i in range(2, size-2):
        smooth_data[i] = np.average([data[i], data[i-1], data[i+1]], axis=0)
    return smooth_data

In [7]:
x_train_set = SmoothData(x_train_set)
x_test_set = SmoothData(x_test_set)

### One-hot encoding

In [9]:
target_onehot  = []
y_test_set_onehot = []
for i in range(len(target)):
    if(target[i]==0):
        target_onehot.append(np.array([1,0,0]))
    elif (target[i]==1):
        target_onehot.append(np.array([0,1,0]))
    elif(target[i]==2):
        target_onehot.append(np.array([0,0,1]))
        
for i in range(len(y_test_set)):
    if(y_test_set[i]==0):
        y_test_set_onehot.append(np.array([1,0,0]))
    elif (y_test_set[i]==1):
        y_test_set_onehot.append(np.array([0,1,0]))
    elif (y_test_set[i]==2):
        y_test_set_onehot.append(np.array([0,0,1]))

### LSTM one-hot

In [239]:
shuffle_x_train = []
shuffle_target = []
shuffle_index = random.sample(range(0, 139000), 2780)
for i in shuffle_index:
    shuffle_x_train.append (np.array([x_train_set[i:i+100]]).reshape(100,3))
    shuffle_target.append (np.array([target_onehot[i:i+100]]).reshape(100,3))

In [85]:
len(x_test_set)

34427

In [240]:
model = Sequential()
model.add(LSTM(10, input_shape = (100,3), return_sequences = True))
model.add(LSTM(10, return_sequences= True))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='RMSprop',metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100, 10)           560       
_________________________________________________________________
lstm_6 (LSTM)                (None, 100, 10)           840       
_________________________________________________________________
dense_3 (Dense)              (None, 100, 3)            33        
Total params: 1,433
Trainable params: 1,433
Non-trainable params: 0
_________________________________________________________________


In [84]:
history = model.fit( 
    np.array([x_train_set[i:i+200] for i in range(0,104000,100)]),
    np.array([target_onehot[i:i+200] for i in range(0,104000,100)]),
    epochs=200,
    validation_data = (np.array([x_train_set[i:i+200] for i in range(104000,105000,50)]),
                     np.array([target_onehot[i:i+200] for i in range(104000,105000,50)])),
    batch_size=64)

Train on 1040 samples, validate on 20 samples
Epoch 1/200
1040/1040 [==============================] - 8s 8ms/step - loss: 0.9931 - categorical_accuracy: 0.8731 - val_loss: 0.7982 - val_categorical_accuracy: 0.8890
Epoch 2/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.5838 - categorical_accuracy: 0.9020 - val_loss: 0.5130 - val_categorical_accuracy: 0.8890
Epoch 3/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.4454 - categorical_accuracy: 0.9020 - val_loss: 0.4865 - val_categorical_accuracy: 0.8890
Epoch 4/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.4238 - categorical_accuracy: 0.9020 - val_loss: 0.4529 - val_categorical_accuracy: 0.8890
Epoch 5/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.4064 - categorical_accuracy: 0.9020 - val_loss: 0.4342 - val_categorical_accuracy: 0.8890
Epoch 6/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.3918 - categorical_accuracy: 0

Epoch 49/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2728 - categorical_accuracy: 0.9106 - val_loss: 0.4706 - val_categorical_accuracy: 0.9105
Epoch 50/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2795 - categorical_accuracy: 0.9111 - val_loss: 0.4215 - val_categorical_accuracy: 0.9105
Epoch 51/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2779 - categorical_accuracy: 0.9104 - val_loss: 0.3433 - val_categorical_accuracy: 0.9107
Epoch 52/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2750 - categorical_accuracy: 0.9111 - val_loss: 0.3881 - val_categorical_accuracy: 0.9115
Epoch 53/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2742 - categorical_accuracy: 0.9105 - val_loss: 0.3949 - val_categorical_accuracy: 0.9112
Epoch 54/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2773 - categorical_accuracy: 0.9113 - val_loss: 0.3984 - val_categoric

1040/1040 [==============================] - 4s 4ms/step - loss: 0.2392 - categorical_accuracy: 0.9159 - val_loss: 0.4151 - val_categorical_accuracy: 0.9183
Epoch 146/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2400 - categorical_accuracy: 0.9169 - val_loss: 0.3330 - val_categorical_accuracy: 0.9180
Epoch 147/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2427 - categorical_accuracy: 0.9159 - val_loss: 0.3807 - val_categorical_accuracy: 0.9178
Epoch 148/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2385 - categorical_accuracy: 0.9192 - val_loss: 0.3448 - val_categorical_accuracy: 0.9185
Epoch 149/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2437 - categorical_accuracy: 0.9184 - val_loss: 0.3852 - val_categorical_accuracy: 0.9165
Epoch 150/200
1040/1040 [==============================] - 4s 4ms/step - loss: 0.2377 - categorical_accuracy: 0.9204 - val_loss: 0.3817 - val_categorical_accur

Epoch 193/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2326 - categorical_accuracy: 0.9244 - val_loss: 0.3792 - val_categorical_accuracy: 0.9165
Epoch 194/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2301 - categorical_accuracy: 0.9247 - val_loss: 0.3871 - val_categorical_accuracy: 0.9180
Epoch 195/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2339 - categorical_accuracy: 0.9236 - val_loss: 0.3722 - val_categorical_accuracy: 0.9165
Epoch 196/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2288 - categorical_accuracy: 0.9247 - val_loss: 0.3822 - val_categorical_accuracy: 0.9170
Epoch 197/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2342 - categorical_accuracy: 0.9247 - val_loss: 0.3479 - val_categorical_accuracy: 0.9178
Epoch 198/200
1040/1040 [==============================] - 5s 5ms/step - loss: 0.2331 - categorical_accuracy: 0.9250 - val_loss: 0.3446 - val_cat

In [241]:
history = model.fit(np.array(shuffle_x_train),
                    np.array(shuffle_target),
                    epochs=200,
                   validation_data = (np.array([x_train_set[i:i+100] for i in range(139000,140000,50)]),
                     np.array([target_onehot[i:i+100] for i in range(139000,140000,50)])),
    batch_size=64)


Train on 2780 samples, validate on 20 samples
Epoch 1/200
2780/2780 [==============================] - 9s 3ms/step - loss: 0.7284 - categorical_accuracy: 0.8811 - val_loss: 0.5829 - val_categorical_accuracy: 0.7990
Epoch 2/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.4361 - categorical_accuracy: 0.8974 - val_loss: 0.6139 - val_categorical_accuracy: 0.7990
Epoch 3/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.4186 - categorical_accuracy: 0.8974 - val_loss: 0.6130 - val_categorical_accuracy: 0.7990
Epoch 4/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.4086 - categorical_accuracy: 0.8974 - val_loss: 0.6049 - val_categorical_accuracy: 0.7990
Epoch 5/200
2780/2780 [==============================] - 7s 2ms/step - loss: 0.3991 - categorical_accuracy: 0.8974 - val_loss: 0.5988 - val_categorical_accuracy: 0.7990
Epoch 6/200
2780/2780 [==============================] - 7s 3ms/step - loss: 0.3861 - categorical_accuracy: 0

Epoch 49/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2781 - categorical_accuracy: 0.9048 - val_loss: 0.5469 - val_categorical_accuracy: 0.8610
Epoch 50/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2798 - categorical_accuracy: 0.9046 - val_loss: 0.5926 - val_categorical_accuracy: 0.8585
Epoch 51/200
2780/2780 [==============================] - 7s 3ms/step - loss: 0.2789 - categorical_accuracy: 0.9049 - val_loss: 0.5744 - val_categorical_accuracy: 0.8615
Epoch 52/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2779 - categorical_accuracy: 0.9050 - val_loss: 0.5905 - val_categorical_accuracy: 0.8575
Epoch 53/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2776 - categorical_accuracy: 0.9055 - val_loss: 0.5880 - val_categorical_accuracy: 0.8585
Epoch 54/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2795 - categorical_accuracy: 0.9060 - val_loss: 0.6026 - val_categoric

2780/2780 [==============================] - 6s 2ms/step - loss: 0.2668 - categorical_accuracy: 0.9102 - val_loss: 0.5124 - val_categorical_accuracy: 0.8755
Epoch 146/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2663 - categorical_accuracy: 0.9103 - val_loss: 0.5038 - val_categorical_accuracy: 0.8755
Epoch 147/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2666 - categorical_accuracy: 0.9106 - val_loss: 0.5117 - val_categorical_accuracy: 0.8760
Epoch 148/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2676 - categorical_accuracy: 0.9084 - val_loss: 0.5120 - val_categorical_accuracy: 0.8740
Epoch 149/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2666 - categorical_accuracy: 0.9099 - val_loss: 0.4999 - val_categorical_accuracy: 0.8765
Epoch 150/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2664 - categorical_accuracy: 0.9092 - val_loss: 0.5231 - val_categorical_accur

Epoch 193/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2633 - categorical_accuracy: 0.9123 - val_loss: 0.5080 - val_categorical_accuracy: 0.8640
Epoch 194/200
2780/2780 [==============================] - 7s 2ms/step - loss: 0.2638 - categorical_accuracy: 0.9112 - val_loss: 0.5084 - val_categorical_accuracy: 0.8640
Epoch 195/200
2780/2780 [==============================] - 7s 2ms/step - loss: 0.2628 - categorical_accuracy: 0.9119 - val_loss: 0.5184 - val_categorical_accuracy: 0.8640
Epoch 196/200
2780/2780 [==============================] - 8s 3ms/step - loss: 0.2630 - categorical_accuracy: 0.9125 - val_loss: 0.5246 - val_categorical_accuracy: 0.8645
Epoch 197/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2630 - categorical_accuracy: 0.9127 - val_loss: 0.5074 - val_categorical_accuracy: 0.8645
Epoch 198/200
2780/2780 [==============================] - 6s 2ms/step - loss: 0.2628 - categorical_accuracy: 0.9119 - val_loss: 0.5175 - val_cat

In [14]:
results = model.predict(np.array(x_test_set[:34400]).reshape(172,200,3))
results = results.reshape(34400,3)

In [37]:
len(x_train_set) + len(x_test_set)

140532

array([[0.68055487, 0.18456206, 0.134883  ],
       [0.8326388 , 0.10047033, 0.06689084],
       [0.90336925, 0.0612621 , 0.0353686 ],
       ...,
       [0.994238  , 0.00386085, 0.00190116],
       [0.99424654, 0.00385065, 0.00190275],
       [0.99425405, 0.00384028, 0.00190558]], dtype=float32)

### Post-processing Section

In [22]:
result_class = []
for i in results:
    if(np.argmax(i)==0):
        result_class.append(0)
    elif(np.argmax(i)==1):
        result_class.append(1)
    elif(np.argmax(i)==2):
        result_class.append(2)

In [38]:
"""
Creating list of events and 
removing events with duration lower than 50

Event tuple: [type, duration]
UPDATED:

Event tuple : [type, init-point, end-point, duration, energy/duration(sec)]
UPDATED:
Event tuple : [type, init-point, end-point, duration, energy/duration(sec), mean(3-max value)]


"""

brks = 0
accs = 0
num_events = 0
events = []
freq = 100 

for i in range(len(result_class)):
    if(result_class[i]==1):
        brks += 1
    elif(result_class[i]==2):
        accs += 1
    else:
        if(brks > 50):
            events.append(("brake", i-brks, i-1, brks/freq))
            print(num_events, "- braking event," ,"duration:", brks/freq, " tsteps", "init:", i-brks ,"end:", i-1)
            brks = 0
            num_events += 1
                  
        elif(accs > 50):
            events.append(("acc", i-accs, i-1, accs/freq))
            print(num_events, "- acceleration event," ,"duration:", accs/freq, " tsteps", "init:", i-accs ,"end:", i-1)
            accs = 0
            num_events += 1

0 - acceleration event, duration: 1.24  tsteps init: 8075 end: 8198
1 - acceleration event, duration: 1.1  tsteps init: 11471 end: 11580
2 - acceleration event, duration: 1.43  tsteps init: 12056 end: 12198
3 - acceleration event, duration: 1.89  tsteps init: 23210 end: 23398
4 - braking event, duration: 0.54  tsteps init: 23690 end: 23743
5 - braking event, duration: 2.06  tsteps init: 26593 end: 26798
6 - braking event, duration: 0.67  tsteps init: 29629 end: 29695
7 - braking event, duration: 0.75  tsteps init: 30324 end: 30398
8 - acceleration event, duration: 1.63  tsteps init: 30636 end: 30798
9 - braking event, duration: 1.87  tsteps init: 31005 end: 31191
10 - acceleration event, duration: 0.98  tsteps init: 31501 end: 31598
11 - braking event, duration: 0.81  tsteps init: 32647 end: 32727


In [29]:
"""
Creating list of events and 
removing events with energy < 20 and duration lower than 30

"""

brks = 0
accs = 0
num_events = 0
events = []
freq = 100 
scaled_x_test = Scale(x_test_set)

for i in range(len(result_class)):
    if(result_class[i]==1):
        brks += 1
    elif(result_class[i]==2):
        accs += 1
    else:
        if(brks > 50 ):
            signal = [abs ((scaled_x_test)[i][0]) for i in range(i-brks, i-1)]
            energy = Energy(signal)
            if(energy > 10):
                events.append(("brake", i-brks, i-1, brks/freq))
                print(num_events, "- braking event," ,"duration:", brks/freq, " tsteps", "init:", i-brks ,"end:", i-1)
                num_events += 1
            brks = 0
        elif(accs > 50):
            signal = [abs ((scaled_x_test)[i][0]) for i in range(i-accs, i-1)]
            energy = Energy(signal)
            if(energy > 10):
                events.append(("acc", i-accs, i-1, accs/freq))
                print(num_events, "- acceleration event," ,"duration:", accs/freq, " tsteps", "init:", i-accs ,"end:", i-1)
                num_events += 1
            accs = 0

0 - acceleration event, duration: 1.25  tsteps init: 8075 end: 8199
1 - acceleration event, duration: 1.11  tsteps init: 11471 end: 11581
2 - acceleration event, duration: 1.44  tsteps init: 12056 end: 12199
3 - acceleration event, duration: 1.9  tsteps init: 23210 end: 23399
4 - braking event, duration: 0.55  tsteps init: 23690 end: 23744
5 - braking event, duration: 2.07  tsteps init: 26593 end: 26799
6 - braking event, duration: 0.68  tsteps init: 29629 end: 29696
7 - braking event, duration: 0.76  tsteps init: 30324 end: 30399
8 - acceleration event, duration: 1.64  tsteps init: 30636 end: 30799
9 - braking event, duration: 1.88  tsteps init: 31005 end: 31192
10 - acceleration event, duration: 0.99  tsteps init: 31501 end: 31599
11 - braking event, duration: 0.82  tsteps init: 32647 end: 32728


In [39]:
"""
Assign intensity to each event 
0, 1, 2 --> Low, Medium, High
thresholds:
    0.98
    1.96
    3.92
"""    
i = 0
scaled_x_test = Scale(x_test_set)
for e in events:
    event_type, init, end, duration = e
    signal = [abs(scaled_x_test)[i][0] for i in range(init, end)]
    events[i] += (ComputeIntensity(signal),)
    i +=1

In [40]:
result_class = np.zeros(len(x_test_set))
for e in events:
    event_type, init, end, duration, intensity = e
    if(event_type =="brake"):
        result_class[init:end] = 1
    elif(event_type=="acc"):
        result_class[init:end] = 2

In [31]:
def ComputeIntensity(signal):
    max_values = np.mean(nlargest(3, signal))
    if(max_values > 0.98 and max_values < 1.96):
        return 0
    elif(max_values > 1.96 and max_values< 3.92):
        return 1
    elif(max_values > 3.92):
        return 2  

In [25]:
"""
Computing the energy (on x-axis)
First, scale the accelerometer signal to the 9.8 
Second, iterate list of events, calculate energy of the x-axis 
"""

def Energy(data):
    data = [abs(number) for number in data]
    return np.sum(np.power(data,2))

In [26]:
"""
k-max value
mean of the k max value 
"""

i = 0
for e in events:
    event_type, init, end, duration, _ = e
    signal = [abs(scaled_x_test[i][0]) for i in range(init, end)]
    print(event_type, np.mean(nlargest(3, signal)))
    i +=1

In [27]:
"""
Scaling signal to m/s2
"""
def Scale(data):
    scaled_x_test = data.copy()
    for i in range(len(scaled_x_test)):
        for j in range(len(scaled_x_test[0])):
            scaled_x_test[i][j] = scaled_x_test[i][j] * 9.8
    return scaled_x_test

In [177]:
"""
Adding energy, max-values attr
"""

i = 0
for e in events:
    event_type, init, end, duration = e
    signal = [abs (scaled_x_test[i][0]) for i in range(init, end)]
    events[i] += (Energy(signal),)
    events[i] += (np.mean(nlargest(3, signal)),)
    i +=1

In [253]:
len(events)

32

In [61]:
scaled_x_train = x_train_set.copy()
for i in range(len(scaled_x_train)):
    for j in range(len(scaled_x_train[0])):
        scaled_x_train[i][j] = scaled_x_train[i][j] * 9.8


print(Energy(scaled_x_test[11027:11228])/2)
print(Energy(scaled_x_train[11634:11735])/2)
print(Energy(scaled_x_train[13151:13454])/3)
print(Energy(scaled_x_train[20838:21141])/3)
print(Energy(scaled_x_train[21346:21546])/2)

182.2395899392445
9.350331882106548
17.11265124272187
32.10789721846348
31.959289845960992


In [46]:
# test the energy for driverbehavior dataset 21 aggressive brake
accLinear21 = pandas.read_csv('./data/21/aceleracaoLinear_terra.csv')
groundTruth21 = pandas.read_csv('./data/21/groundTruthEdited.csv')

accLinear21 = accLinear21[['x','y','z']]
accLinear21 = np.array(accLinear21)
accLinear21 = SmoothData(accLinear21)



In [151]:
print(Energy(accLinear21[10105:10256])/(151/50))
print(Energy(accLinear21[13059:13261])/4  )
print(Energy(accLinear21[14689:14892])/4)
print(Energy(accLinear21[16421:16573])/3)
print(Energy(accLinear21[17287:17490])/4)
print(Energy(accLinear21[19834:19986])/3)
print(Energy(accLinear21[9595:9849])/5)
print(Energy(accLinear21[1751:1954])/4)
# print(np.mean(nlargest(3,[abs(accLinear21[i][0]) for i in range(10105,10256)])))
# print(np.mean(nlargest(3,[abs(accLinear21[i][0]) for i in range(13059,13261)])))
# print(np.mean(nlargest(3,[abs(accLinear21[i][0]) for i in range(14689,14892)])))
# print(np.mean(nlargest(3,[abs(accLinear21[i][0]) for i in range(16421,16573)])))
# print(np.mean(nlargest(3,[abs(accLinear21[i][0]) for i in range(17287,17490)])))
# print(np.mean(nlargest(3,[abs(accLinear21[i][0]) for i in range(19834,19986)])))

# print(Energy(accLinear21[13059:13261])/4)
# print(Energy(accLinear21[14689:14892])/4)
# print(Energy(accLinear21[16421:16573])/3)
# print(Energy(accLinear21[17287:17490])/4)
# print(Energy(accLinear21[19834:19986])/3)
# print(Energy(accLinear21[9595:9849])/5)
# print(Energy(accLinear21[1751:1954])/4)



653.6227058011023
482.4623294334049
481.1426101592832
522.5912234803596
152.52759937798353
534.6667930646526
41.06582885307229
23.175237433004195


In [35]:
from sklearn.metrics import confusion_matrix

y_true = y_test_set.copy()
y_pred = result_class.copy()
classes = [0,1,2]
confusion_matrix(y_true, y_pred, labels=[0,1,2])

array([[29619,   316,   270],
       [ 1619,   354,     0],
       [ 1692,     0,   557]], dtype=int64)

In [47]:
groundTruth21

,evento,startIndexinAcc,endIndexinAcc
0,evento_nao_agressivo,1751,1954
1,troca_faixa_esquerda_agressiva,1140,1241
2,evento_nao_agressivo,4298,4552
3,troca_faixa_esquerda_agressiva,4909,5112
4,troca_faixa_esquerda_agressiva,5470,5622
5,evento_nao_agressivo,5928,6182
6,troca_faixa_esquerda_agressiva,8271,8423
7,evento_nao_agressivo,9595,9849
8,freada_agressiva,10105,10256
9,freada_agressiva,13059,13261


### Graph the results

In [36]:

fig =tools.make_subplots(rows=1,cols=1)

result =  go.Scatter(
    x = list(range(0,34400)),
    y = result_class,
    mode = 'lines',
    name='model_result'
)

test =  go.Scatter(
    x = list(range(0,34400)),
    y = y_test_set,
    mode = 'lines',
    name='test'
)

fig.append_trace(result,1,1)
fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Brake_Acc Model Classification')


py.offline.plot(fig, filename='my_data_brake_acc_onehot_200_softmax_categorical_crossentropy')


C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `my_data_brake_acc_onehot_200_softmax_categorical_crossentropy` didn't end with .html. Adding .html to the end of your file.



'my_data_brake_acc_onehot_200_softmax_categorical_crossentropy.html'

In [238]:
events

[('acc', 2021, 2168, 1.48, 1),
 ('acc', 4094, 4199, 1.06, 0),
 ('acc', 5324, 5399, 0.76, 1),
 ('acc', 5421, 5599, 1.79, 1),
 ('acc', 5618, 5729, 1.12, 1),
 ('acc', 6796, 6999, 2.04, 1),
 ('acc', 7017, 7168, 1.52, 1),
 ('acc', 7949, 7999, 0.51, 1),
 ('acc', 8020, 8199, 1.8, 0),
 ('acc', 8244, 8365, 1.22, 0),
 ('acc', 13070, 13199, 1.3, 1),
 ('acc', 17992, 18199, 2.08, 1),
 ('brake', 18360, 18530, 1.71, 1),
 ('acc', 21449, 21599, 1.51, 0),
 ('acc', 21675, 21799, 1.25, 0),
 ('acc', 25275, 25360, 0.86, None),
 ('acc', 26616, 26794, 1.79, 1),
 ('acc', 27239, 27379, 1.41, 0)]

In [61]:

# test = pandas.read_csv('./data/Learning/brake, acc/Test_Set.csv')
test = pandas.read_csv('./data/Learning/MyData/version1/Test_Set.csv')


fig =tools.make_subplots(rows=1,cols=1)


test =  go.Scatter(
    x = list(range(0,27100)),
    y = test['x'],
    mode = 'lines',
    name='test'
)

fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Test Data')


py.offline.plot(fig, filename='brake_acc_test_data')


C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `brake_acc_test_data` didn't end with .html. Adding .html to the end of your file.



'brake_acc_test_data.html'

In [254]:
fig =tools.make_subplots(rows=2,cols=1)

loss_result =  go.Scatter(
    x = list(range(0,500)),
    y = history.history['loss'],
    mode = 'lines'
)

acc_result =  go.Scatter(
    x = list(range(0,500)),
    y = history.history['categorical_accuracy'],
    mode = 'lines'
)

fig.append_trace(loss_result, 1, 1)
fig.append_trace(acc_result, 2, 1)

fig['layout'].update(height=650, title = 'Loss, Accuracy')
py.offline.plot(fig, filename='multiple-axes-double')

C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `multiple-axes-double` didn't end with .html. Adding .html to the end of your file.



'multiple-axes-double.html'

### Saving model

In [97]:
from keras.models import model_from_json
import os

# serialize model to JSON
model_json = model.to_json()
with open("./Learning_model/version1/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./Learning_model/version1/model.h5")
print("Saved model to disk")

Saved model to disk


### Loading Model

In [1]:
from keras.models import model_from_json
import os

# load json and create model
json_file = open('./Learning_model/version1/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Learning_model/version1/model.h5")
print("Loaded model from disk")
model = loaded_model

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Loaded model from disk


### Sensor Log Generated Data

In [106]:
my_data_part3 = pandas.read_csv('./MyData/22.4.2020.part3.csv')
my_data_part2 = pandas.read_csv('./MyData/22.4.2020.part2.csv')
my_data_round1 = pandas.read_csv('./MyData/22.4.2020/round1.csv')
my_data_round2 = pandas.read_csv('./MyData/22.4.2020/round2.csv')
my_data_round3 = pandas.read_csv('./MyData/22.4.2020/round3.csv')


# working_set = my_data[['accelerometerAccelerationX(G)','accelerometerAccelerationY(G)','accelerometerAccelerationZ(G)']][3200:3700]        

set_user_part3 = my_data_part3[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_part2 = my_data_part2[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_round1 = my_data_round1[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_round2 = my_data_round2[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_round3 = my_data_round3[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]

######

my_data_brake_round4 = pandas.read_csv('./MyData/24.4.2020/brake_round4.csv')
my_data_brake_round5 = pandas.read_csv('./MyData/24.4.2020/brake_round5.csv')

set_user_brake_round4 = my_data_brake_round4[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]
set_user_brake_round5 = my_data_brake_round5[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]

In [107]:
def RenderData(data, fig, init, end):
    acx =  go.Scatter(
        x = list(range(init, end)),
        y = [data[i][0] for i in range(init, end)],
        mode = 'lines',
        name = 'x'
    )
    

    acy =  go.Scatter(
        x = list(range(init, end)),
        y = [data[i][1] for i in range(init, end)],
        mode = 'lines',
        name='y'
    )

    acz =  go.Scatter(
        x = list(range(init, end)),
        y = [data[i][2] for i in range(init, end)],
        mode = 'lines',
        name='z'
    )
    fig.append_trace(acx,1,1)
    fig.append_trace(acy,1,1)
    fig.append_trace(acz,1,1)
    
    return fig

In [108]:
set_user_part2 = np.array(set_user_part2)
set_user_part2 = SmoothData(set_user_part2)
set_user_part3 = np.array(set_user_part3)
set_user_part3 = SmoothData(set_user_part3)
######
set_user_round1 = np.array(set_user_round1)
set_user_round2 = np.array(set_user_round2)
set_user_round3 = np.array(set_user_round3)
set_user_round1 = SmoothData(set_user_round1)
set_user_round2 = SmoothData(set_user_round2)
set_user_round3 = SmoothData(set_user_round3)

set_user_brake_round4 = np.array(set_user_brake_round4)
set_user_brake_round5 = np.array(set_user_brake_round5)
set_user_brake_round4 = SmoothData(set_user_brake_round4)
set_user_brake_round5 = SmoothData(set_user_brake_round5)

In [162]:
fig =tools.make_subplots(rows=1,cols=1)
RenderData(set_user_round1, fig,  8093, 8395)
fig['layout'].update(height=500, title = 'Test Data')
py.offline.plot(fig, filename='brake_acc_test_data')

C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `brake_acc_test_data` didn't end with .html. Adding .html to the end of your file.



'brake_acc_test_data.html'

In [159]:
len(set_user_brake_round5)

47371